In [1]:
!pip install opencv-python boto3 botocore ffmpeg-python moviepy --upgrade -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.25.63 requires botocore==1.27.62, but you have botocore 1.27.77 which is incompatible.
awscli 1.25.63 requires PyYAML<5.5,>=3.10, but you have pyyaml 6.0 which is incompatible.
awscli 1.25.63 requires rsa<4.8,>=3.1.2, but you have rsa 4.9 which is incompatible.
aiobotocore 2.3.4 requires botocore<1.24.22,>=1.24.21, but you have botocore 1.27.77 which is incompatible.


In [11]:
import json
import logging
import os
import urllib

import boto3
import botocore
# import cv2
import numpy as np

In [40]:
from blur_faces.helpers import get_timestamps_and_faces

In [41]:
from blur_faces.rekognition import boto3_client, start_face_detection

In [42]:
from blur_faces.helpers import wait_for_completion

In [44]:
bucket = 'datasets-dggallit'
video = 'rekognition-video-demo/video-test.mp4'
local_path_to_video = 'videos/video-test.mp4'
local_output = 'videos/output.mp4'

reko = boto3_client()
job_id = start_face_detection(bucket, video, 1, reko)
response = wait_for_completion(job_id, reko_client=reko)
timestamps=get_timestamps_and_faces(response, job_id, reko)
apply_faces_to_video(timestamps, local_path_to_video, local_output, response["VideoMetadata"])
# integrate_audio('videos/video-test.mp4', 'videos/output.mp4')

AccessDeniedException: An error occurred (AccessDeniedException) when calling the StartFaceDetection operation: User: arn:aws:sts::232838030412:assumed-role/AmazonSageMaker-ExecutionRole-20211204T182243/SageMaker is not authorized to perform: rekognition:StartFaceDetection because no identity-based policy allows the rekognition:StartFaceDetection action